In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install -U bitsandbytes

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

2026-01-15 13:01:56.568001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768482116.589964     170 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768482116.596530     170 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768482116.613673     170 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768482116.613692     170 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768482116.613694     170 computation_placer.cc:177] computation placer alr

In [6]:
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
ADAPTER_PATH = "/kaggle/input/model/transformers/default/1/adapters"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [11]:
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [13]:
model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH
)

model = model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [14]:
model.save_pretrained("model_fp16")
tokenizer.save_pretrained("model_fp16")

('model_fp16/tokenizer_config.json',
 'model_fp16/special_tokens_map.json',
 'model_fp16/chat_template.jinja',
 'model_fp16/tokenizer.model',
 'model_fp16/added_tokens.json',
 'model_fp16/tokenizer.json')

In [15]:
model_int8 = AutoModelForCausalLM.from_pretrained(
    "model_fp16",
    load_in_8bit=True,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [16]:
model_int8.save_pretrained("quantized/model-int8")
tokenizer.save_pretrained("quantized/model-int8")

('quantized/model-int8/tokenizer_config.json',
 'quantized/model-int8/special_tokens_map.json',
 'quantized/model-int8/chat_template.jinja',
 'quantized/model-int8/tokenizer.model',
 'quantized/model-int8/added_tokens.json',
 'quantized/model-int8/tokenizer.json')

In [17]:
model_int4 = AutoModelForCausalLM.from_pretrained(
    "model_fp16",
    load_in_4bit=True,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [19]:
model_int4.save_pretrained("quantized/model-int4")
tokenizer.save_pretrained("quantized/model-int4")

('quantized/model-int4/tokenizer_config.json',
 'quantized/model-int4/special_tokens_map.json',
 'quantized/model-int4/chat_template.jinja',
 'quantized/model-int4/tokenizer.model',
 'quantized/model-int4/added_tokens.json',
 'quantized/model-int4/tokenizer.json')

In [20]:
!du -sh model_fp16
!du -sh quantized/model-int8
!du -sh quantized/model-int4

2.1G	model_fp16
1.2G	quantized/model-int8
774M	quantized/model-int4


In [21]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 76096, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 76096 (delta 76), reused 35 (delta 35), pack-reused 75935 (from 2)
Receiving objects: 100% (76096/76096), 279.86 MiB | 41.92 MiB/s, done.
Resolving deltas: 100% (55208/55208), done.


In [22]:
cd llama.cpp

/kaggle/working/llama.cpp


In [23]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 119.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 104.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━

In [25]:
!python convert_hf_to_gguf.py \/kaggle/working/model_fp16 \--outfile ../quantized/model.gguf \--outtype f16

INFO:hf-to-gguf:Loading model: model_fp16
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:output.weight,               torch.float16 --> F16, shape = {2048, 32000}
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {2048, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.attn_output.weig

In [36]:
!cd build && cmake .. && cmake --build . --config Release

CMAKE_BUILD_TYPE=Release
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- x86 detected
-- Adding CPU backend variant ggml-cpu: -march=native 
-- ggml version: 0.9.5
-- ggml commit:  8cc0ba957
-- OpenSSL found: 3.0.2
-- Generating embedded license file for target: common
-- Configuring done (0.4s)
-- Generating done (0.3s)
-- Build files have been written to: /kaggle/working/llama.cpp/build
[  0%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml.c.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml.cpp.o
[  1%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml-alloc.c.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-backend.cpp.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-opt.cpp.o
[  2%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-threading.c

In [37]:
%cd ..
!cd llama.cpp/build/bin && \.//llama-quantize \/kaggle/working/quantized/model.gguf \/kaggle/working/quantized/model-q4_0.gguf \q4_0 

/kaggle/working
main: build = 7744 (8cc0ba957)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing '/kaggle/working/quantized/model.gguf' to '/kaggle/working/quantized/model-q4_0.gguf' as Q4_0
llama_model_loader: direct I/O is enabled, disabling mmap
llama_model_loader: loaded meta data with 32 key-value pairs and 201 tensors from /kaggle/working/quantized/model.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Model_Fp16
llama_model_loader: - kv   3:                         general.size_label str              = 1.1B
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llam

In [38]:
!zip -r day3.zip model_fp16 quantized/model-int8 quantized/model-int4

  adding: model_fp16/ (stored 0%)
  adding: model_fp16/special_tokens_map.json (deflated 79%)
  adding: model_fp16/generation_config.json (deflated 29%)
  adding: model_fp16/chat_template.jinja (deflated 60%)
  adding: model_fp16/tokenizer.json (deflated 85%)
  adding: model_fp16/tokenizer_config.json (deflated 69%)
  adding: model_fp16/tokenizer.model (deflated 55%)
  adding: model_fp16/config.json (deflated 48%)
  adding: model_fp16/model.safetensors (deflated 22%)
  adding: quantized/model-int8/ (stored 0%)
  adding: quantized/model-int8/special_tokens_map.json (deflated 79%)
  adding: quantized/model-int8/generation_config.json (deflated 29%)
  adding: quantized/model-int8/chat_template.jinja (deflated 60%)
  adding: quantized/model-int8/tokenizer.json (deflated 85%)
  adding: quantized/model-int8/tokenizer_config.json (deflated 69%)
  adding: quantized/model-int8/tokenizer.model (deflated 55%)
  adding: quantized/model-int8/config.json (deflated 56%)
  adding: quantized/model-int8